In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install pyspark 

     |████████████████████████████████| 212.3MB 81kB/s 
     |████████████████████████████████| 204kB 22.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=e897226554875f63fbb4481c3acc3296fd710a0df24250529ae9115d69a80f56
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Aggriculture').getOrCreate()
df = spark.read.csv('train_yaOffsB.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Estimated_Insects_Count: integer (nullable = true)
 |-- Crop_Type: integer (nullable = true)
 |-- Soil_Type: integer (nullable = true)
 |-- Pesticide_Use_Category: integer (nullable = true)
 |-- Number_Doses_Week: integer (nullable = true)
 |-- Number_Weeks_Used: integer (nullable = true)
 |-- Number_Weeks_Quit: integer (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Crop_Damage: integer (nullable = true)



In [4]:
df.show(10)

+---------+-----------------------+---------+---------+----------------------+-----------------+-----------------+-----------------+------+-----------+
|       ID|Estimated_Insects_Count|Crop_Type|Soil_Type|Pesticide_Use_Category|Number_Doses_Week|Number_Weeks_Used|Number_Weeks_Quit|Season|Crop_Damage|
+---------+-----------------------+---------+---------+----------------------+-----------------+-----------------+-----------------+------+-----------+
|F00000001|                    188|        1|        0|                     1|                0|                0|                0|     1|          0|
|F00000003|                    209|        1|        0|                     1|                0|                0|                0|     2|          1|
|F00000004|                    257|        1|        0|                     1|                0|                0|                0|     2|          1|
|F00000005|                    257|        1|        1|                     1|          

In [5]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
ID,F00000001,F00000003,F00000004,F00000005,F00000006
Estimated_Insects_Count,188,209,257,257,342
Crop_Type,1,1,1,1,1
Soil_Type,0,0,0,1,0
Pesticide_Use_Category,1,1,1,1,1
Number_Doses_Week,0,0,0,0,0
Number_Weeks_Used,0,0,0,0,0
Number_Weeks_Quit,0,0,0,0,0
Season,1,2,2,2,2
Crop_Damage,0,1,1,1,1


In [6]:
df.columns

['ID',
 'Estimated_Insects_Count',
 'Crop_Type',
 'Soil_Type',
 'Pesticide_Use_Category',
 'Number_Doses_Week',
 'Number_Weeks_Used',
 'Number_Weeks_Quit',
 'Season',
 'Crop_Damage']

In [7]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
ID,88858,None,None,F00000001,F00155945
Estimated_Insects_Count,88858,1399.0122104931463,849.0487806668847,150,4097
Crop_Type,88858,0.2843750703369421,0.4511188091519747,0,1
Soil_Type,88858,0.4584167998379437,0.49827064083043343,0,1
Pesticide_Use_Category,88858,2.2641855544801817,0.46177158158329773,1,3
Number_Doses_Week,88858,25.849951608183844,15.554427765396934,0,95
Number_Weeks_Used,79858,28.62397004683313,12.391881462463685,0,67
Number_Weeks_Quit,88858,9.589986270228904,9.900631115271118,0,50
Season,88858,1.8969591933196785,0.701322132743697,1,3


In [8]:
columns=['ID','Estimated_Insects_Count','Crop_Type','Soil_Type','Pesticide_Use_Category','Number_Doses_Week','Number_Weeks_Used',
         'Number_Weeks_Quit','Season','Crop_Damage']

In [9]:
df.toPandas()[columns].nunique()

ID                         88858
Estimated_Insects_Count       71
Crop_Type                      2
Soil_Type                      2
Pesticide_Use_Category         3
Number_Doses_Week             20
Number_Weeks_Used             64
Number_Weeks_Quit             51
Season                         3
Crop_Damage                    3
dtype: int64

In [10]:
numeric_feautres=['Estimated_Insects_Count','Number_Weeks_Used','Number_Weeks_Quit']

categorical_feautres=['Crop_Type','Soil_Type','Pesticide_Use_Category','Number_Doses_Week','Season']

target=['Crop_Damage']

In [11]:
numeric_data = df.select(numeric_feautres).toPandas()

In [12]:
df=df.select('Estimated_Insects_Count','Crop_Type','Soil_Type','Pesticide_Use_Category','Number_Doses_Week','Number_Weeks_Used',
         'Number_Weeks_Quit','Season','Crop_Damage')

cols = df.columns
df.printSchema()

root
 |-- Estimated_Insects_Count: integer (nullable = true)
 |-- Crop_Type: integer (nullable = true)
 |-- Soil_Type: integer (nullable = true)
 |-- Pesticide_Use_Category: integer (nullable = true)
 |-- Number_Doses_Week: integer (nullable = true)
 |-- Number_Weeks_Used: integer (nullable = true)
 |-- Number_Weeks_Quit: integer (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Crop_Damage: integer (nullable = true)



In [13]:
df.toPandas().isna().sum()

Estimated_Insects_Count       0
Crop_Type                     0
Soil_Type                     0
Pesticide_Use_Category        0
Number_Doses_Week             0
Number_Weeks_Used          9000
Number_Weeks_Quit             0
Season                        0
Crop_Damage                   0
dtype: int64

In [14]:
df.filter(df['Number_Weeks_Used'].isNull()).count()

9000

In [15]:
df.columns

['Estimated_Insects_Count',
 'Crop_Type',
 'Soil_Type',
 'Pesticide_Use_Category',
 'Number_Doses_Week',
 'Number_Weeks_Used',
 'Number_Weeks_Quit',
 'Season',
 'Crop_Damage']

In [16]:
for j in ['Estimated_Insects_Count','Crop_Type','Soil_Type','Pesticide_Use_Category','Number_Doses_Week','Number_Weeks_Used','Number_Weeks_Quit']:
    print("Correlation with"+j+" Crop_Damage",df.corr('Crop_Damage',j))

Correlation withEstimated_Insects_Count Crop_Damage 0.2028883265043235
Correlation withCrop_Type Crop_Damage -0.016494705652070066
Correlation withSoil_Type Crop_Damage -0.021997272041923074
Correlation withPesticide_Use_Category Crop_Damage 0.17500204470503275
Correlation withNumber_Doses_Week Crop_Damage -0.03087425832208959
Correlation withNumber_Weeks_Used Crop_Damage 0.1746679539287501
Correlation withNumber_Weeks_Quit Crop_Damage -0.13311659360152783


In [17]:
df.agg({"Number_Weeks_Used":"mean"}).collect()

[Row(avg(Number_Weeks_Used)=28.62397004683313)]

In [18]:
df=df.na.fill(value=28.62397004683313,subset=["Number_Weeks_Used"])

In [19]:
from pyspark.ml.feature import  StringIndexer, VectorAssembler
stages = []
label_stringIdx = StringIndexer(inputCol = 'Crop_Damage', outputCol = 'label')
stages += [label_stringIdx]

assemblerInputs = categorical_feautres + numeric_feautres
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [20]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()


root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- Estimated_Insects_Count: integer (nullable = true)
 |-- Crop_Type: integer (nullable = true)
 |-- Soil_Type: integer (nullable = true)
 |-- Pesticide_Use_Category: integer (nullable = true)
 |-- Number_Doses_Week: integer (nullable = true)
 |-- Number_Weeks_Used: integer (nullable = true)
 |-- Number_Weeks_Quit: integer (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Crop_Damage: integer (nullable = true)



In [21]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
label,0,1,1,1,1
features,"(1.0, 0.0, 1.0, 0.0, 1.0, 188.0, 0.0, 0.0)","(1.0, 0.0, 1.0, 0.0, 2.0, 209.0, 0.0, 0.0)","(1.0, 0.0, 1.0, 0.0, 2.0, 257.0, 0.0, 0.0)","[1.0, 1.0, 1.0, 0.0, 2.0, 257.0, 0.0, 0.0]","(1.0, 0.0, 1.0, 0.0, 2.0, 342.0, 0.0, 0.0)"
Estimated_Insects_Count,188,209,257,257,342
Crop_Type,1,1,1,1,1
Soil_Type,0,0,0,1,0
Pesticide_Use_Category,1,1,1,1,1
Number_Doses_Week,0,0,0,0,0
Number_Weeks_Used,0,0,0,0,0
Number_Weeks_Quit,0,0,0,0,0
Season,1,2,2,2,2


In [22]:
train, test = df.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 62220
Test Dataset Count: 26638


In [23]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)

In [24]:
trainingSummary = lrModel.summary

In [26]:
trainingSummary.accuracy

0.8339279974284796

In [27]:
predictions = lrModel.transform(test)
predictions.select( 'label', 'rawPrediction', 'prediction', 'probability').show(40)

+-----+--------------------+----------+--------------------+
|label|       rawPrediction|prediction|         probability|
+-----+--------------------+----------+--------------------+
|  0.0|[2.15187785121689...|       0.0|[0.90495552620899...|
|  0.0|[1.54684033587725...|       0.0|[0.76775925324628...|
|  0.0|[1.87910031695412...|       0.0|[0.84546558480914...|
|  0.0|[1.84597146164734...|       0.0|[0.83766828973997...|
|  0.0|[1.40701412883248...|       0.0|[0.70287932939929...|
|  0.0|[1.16770173984007...|       0.0|[0.60645786864169...|
|  0.0|[2.32164451372330...|       0.0|[0.91662813020015...|
|  0.0|[2.05041087090433...|       0.0|[0.87068062319702...|
|  0.0|[2.02033335753369...|       0.0|[0.86455129077413...|
|  0.0|[1.90089511603293...|       0.0|[0.83770723457881...|
|  0.0|[1.81327801186631...|       0.0|[0.81534086757758...|
|  0.0|[1.56742703301072...|       0.0|[0.73985879169650...|
|  0.0|[1.40091094449506...|       0.0|[0.67832036208770...|
|  0.0|[1.20039418869085

In [28]:
trainingSummary.precisionByLabel

[0.8412604918274187, 0.4268846503178928, 0.0]

In [32]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 17)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)
predictions.select( 'label', 'rawPrediction', 'prediction', 'probability').show(10)


+-----+--------------+----------+--------------------+
|label| rawPrediction|prediction|         probability|
+-----+--------------+----------+--------------------+
|  0.0| [1.0,5.0,0.0]|       1.0|[0.16666666666666...|
|  0.0| [0.0,6.0,0.0]|       1.0|       [0.0,1.0,0.0]|
|  0.0| [0.0,2.0,0.0]|       1.0|       [0.0,1.0,0.0]|
|  0.0| [1.0,1.0,0.0]|       0.0|       [0.5,0.5,0.0]|
|  0.0| [0.0,0.0,2.0]|       2.0|       [0.0,0.0,1.0]|
|  0.0| [0.0,2.0,0.0]|       1.0|       [0.0,1.0,0.0]|
|  0.0|[3.0,44.0,3.0]|       1.0|    [0.06,0.88,0.06]|
|  0.0| [1.0,1.0,0.0]|       0.0|       [0.5,0.5,0.0]|
|  0.0| [1.0,1.0,0.0]|       0.0|       [0.5,0.5,0.0]|
|  0.0| [0.0,8.0,0.0]|       1.0|       [0.0,1.0,0.0]|
+-----+--------------+----------+--------------------+
only showing top 10 rows



In [43]:
test_df=test.toPandas()
pred_df=predictions.toPandas()

In [45]:
y_test=test_df['Crop_Damage'].values
y_pred=pred_df['prediction'].values


In [46]:
from sklearn.metrics import accuracy_score
print("The Accuracy Score",accuracy_score(y_test,y_pred))


The Accuracy Score 0.7980704257076358
